# Calculating values on total and net peak demand change reported in the text.

Developed by Siobhan Powell, 2022. 

In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.gridspec import GridSpec
import pickle
import datetime

In [3]:
noev_scenario = pd.read_csv('Results/NoEVs_year2035_solar3.5x_wind3x_withstorage_dpdf_20220408.csv')

# Total Demand

In [5]:
scens1 = ['_TimersRandom_noWPcontrol', '_Timers9pm_noWPcontrol', '_Timers12am_noWPcontrol', '_TimersNone_noWPcontrol', '_TimersNone_WPcontrol_minpeak', '_TimersNone_WPcontrol_avgem']
scens2 = ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']

vals = np.zeros((10, 7, 5)) # peak mean weekday total demand
vals2 = np.zeros((10, 7, 5)) # time of day
tables_dfs = {penlevel: pd.DataFrame(np.zeros((7, 5)), 
                                     index=['_TimersRandom_noWPcontrol', '_Timers9pm_noWPcontrol', '_Timers12am_noWPcontrol', '_TimersNone_noWPcontrol', '_TimersNone_WPcontrol_minpeak', '_TimersNone_WPcontrol_avgem', '_TimersMixed_WPcontrol_minpeak'], 
                                     columns=['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork', 'BusinessAsUsual']) for penlevel in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
tables_dfs_timeofpeak = {penlevel: pd.DataFrame(np.zeros((7, 5)), 
                                                index=['_TimersRandom_noWPcontrol', '_Timers9pm_noWPcontrol', '_Timers12am_noWPcontrol', '_TimersNone_noWPcontrol', '_TimersNone_WPcontrol_minpeak', '_TimersNone_WPcontrol_avgem', '_TimersMixed_WPcontrol_minpeak'], 
                                                columns=['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork', 'BusinessAsUsual']) for penlevel in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}

for k, penlevel in enumerate([0.5, 1.0]):#0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]):
#     if penlevel == 0.5:
    folder = 'Fuel1_Solar35_Wind3'
#     else:
#         folder = 'Fuel1_Solar35_Wind3_Curves'
    for i, scen1 in enumerate(scens1):
        for j, scen2 in enumerate(scens2):
            demand_df = pd.read_csv('Results/'+folder+'/fuel1_solar3.5_wind3_'+scen2+scen1+'_penlevel'+str(penlevel)+'_withstorage_dpdf_20220408.csv')
            inds = demand_df[pd.to_datetime(demand_df['datetime']).dt.weekday.isin([0,1,2,3,4])].index
            vals[k, i, j] = demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()
            vals2[k, i, j] = np.where(demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0) == demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max())[0][0]
            tables_dfs[penlevel].loc[scen1, scen2] = demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()
            tables_dfs_timeofpeak[penlevel].loc[scen1, scen2] = np.where(demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0) == demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max())[0][0]
        
    scen2 = 'BusinessAsUsual'
    scen1 =  '_TimersMixed_WPcontrol_minpeak'
    i = i+1
    j = j+1
    demand_df = pd.read_csv('Results/'+folder+'/fuel1_solar3.5_wind3_'+scen2+scen1+'_penlevel'+str(penlevel)+'_withstorage_dpdf_20220408.csv')
    inds = demand_df[pd.to_datetime(demand_df['datetime']).dt.weekday.isin([0,1,2,3,4])].index
    vals[k, i, j] = demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()
    vals2[k, i, j] = np.where(demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0) == demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max())[0][0]
    tables_dfs[penlevel].loc[scen1, scen2] = demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()
    tables_dfs_timeofpeak[penlevel].loc[scen1, scen2] = np.where(demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0) == demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max())[0][0]




In [6]:
vals_shift = noev_scenario.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()

In [7]:
print('Peak total demand increase in 2035 from adding EVs, with 100% is up to:', ((tables_dfs[1.0] - vals_shift) / vals_shift).max().max())
print('Peak total demand increase in 2035 from adding EVs, with 50% is up to:', ((tables_dfs[0.5] - vals_shift) / vals_shift).max().max())

Peak total demand increase in 2035 from adding EVs, with 100% is up to: 0.2592182851018414
Peak total demand increase in 2035 from adding EVs, with 50% is up to: 0.09377994685180023


In [8]:
vals_tmp = (tables_dfs[1.0] - vals_shift) / vals_shift
print('Peak total demand increase in 2035 from adding EVs, with 100%, ranges between:', vals_tmp[vals_tmp>0].min().min(), ((tables_dfs[1.0] - vals_shift) / vals_shift).max().max())
vals_tmp = (tables_dfs[0.5] - vals_shift) / vals_shift
print('Peak total demand increase in 2035 from adding EVs, with 50%, ranges between:', vals_tmp[vals_tmp>0].min().min(), ((tables_dfs[0.5] - vals_shift) / vals_shift).max().max())

Peak total demand increase in 2035 from adding EVs, with 100%, ranges between: 0.09175181096134341 0.2592182851018414
Peak total demand increase in 2035 from adding EVs, with 50%, ranges between: 0.034174271883058036 0.09377994685180023


In [9]:
print('Max value without EVs occurs at 5pm: ', np.round(noev_scenario.loc[pd.to_datetime(noev_scenario['datetime']).dt.weekday.isin([0,1,2,3,4])]['total_incl_noncombustion'].values.reshape(-1, 24).mean(axis=0).max(), 2))

Max value without EVs occurs at 5pm:  109478.38


In [10]:
vals_shift = noev_scenario.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()

print('Percent change at 50% adoption: ')
(tables_dfs[0.5] - vals_shift) / vals_shift

Percent change at 50% adoption: 


,UniversalHome,HighHome,LowHome_HighWork,LowHome_LowWork,BusinessAsUsual
_TimersRandom_noWPcontrol,0.047591,0.034234,0.067746,0.059060,-1.000000
_Timers9pm_noWPcontrol,0.078452,0.038177,0.067691,0.059016,-1.000000
_Timers12am_noWPcontrol,0.045701,0.034174,0.067619,0.058925,-1.000000
_TimersNone_noWPcontrol,0.062278,0.043343,0.067658,0.059110,-1.000000
_TimersNone_WPcontrol_minpeak,0.064510,0.050899,0.065970,0.054285,-1.000000
_TimersNone_WPcontrol_avgem,0.062038,0.042854,0.093780,0.068607,-1.000000
_TimersMixed_WPcontrol_minpeak,-1.000000,-1.000000,-1.000000,-1.000000,0.046369


In [11]:
print('Ranges from _ to _:')
print(np.round(((tables_dfs[0.5] - vals_shift) / vals_shift)[((tables_dfs[0.5] - vals_shift) / vals_shift) > 0].min().min(), 4))
print(np.round(((tables_dfs[0.5] - vals_shift) / vals_shift)[((tables_dfs[0.5] - vals_shift) / vals_shift) > 0].max().max(), 4))

Ranges from _ to _:
0.0342
0.0938


In [12]:
vals_shift = noev_scenario.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()

print('Percent change at 100% adoption: ')
(tables_dfs[1.0] - vals_shift) / vals_shift

Percent change at 100% adoption: 


,UniversalHome,HighHome,LowHome_HighWork,LowHome_LowWork,BusinessAsUsual
_TimersRandom_noWPcontrol,0.121326,0.091752,0.213996,0.196622,-1.000000
_Timers9pm_noWPcontrol,0.237872,0.157322,0.213885,0.196534,-1.000000
_Timers12am_noWPcontrol,0.106010,0.091778,0.213741,0.196352,-1.000000
_TimersNone_noWPcontrol,0.149289,0.101295,0.213819,0.196724,-1.000000
_TimersNone_WPcontrol_minpeak,0.150546,0.107326,0.146388,0.155981,-1.000000
_TimersNone_WPcontrol_avgem,0.149295,0.111010,0.259218,0.208872,-1.000000
_TimersMixed_WPcontrol_minpeak,-1.000000,-1.000000,-1.000000,-1.000000,0.094067


In [14]:
print('Ranges from _ to _:')
print(np.round(((tables_dfs[1.0] - vals_shift) / vals_shift)[((tables_dfs[1.0] - vals_shift) / vals_shift) > 0].min().min(), 4))
print(np.round(((tables_dfs[1.0] - vals_shift) / vals_shift)[((tables_dfs[1.0] - vals_shift) / vals_shift) > 0].max().max(), 4))

Ranges from _ to _:
0.0918
0.2592


In [13]:
print('Hour of peak in mean weekday total demand at 100%:')
tables_dfs_timeofpeak[1.0]

Hour of peak in mean weekday total demand at 100%:


,UniversalHome,HighHome,LowHome_HighWork,LowHome_LowWork,BusinessAsUsual
_TimersRandom_noWPcontrol,20.0,10.0,10.0,10.0,0.0
_Timers9pm_noWPcontrol,21.0,21.0,10.0,10.0,0.0
_Timers12am_noWPcontrol,19.0,10.0,10.0,10.0,0.0
_TimersNone_noWPcontrol,20.0,19.0,10.0,10.0,0.0
_TimersNone_WPcontrol_minpeak,20.0,19.0,10.0,10.0,0.0
_TimersNone_WPcontrol_avgem,20.0,11.0,11.0,11.0,0.0
_TimersMixed_WPcontrol_minpeak,0.0,0.0,0.0,0.0,20.0


# Net Demand

In [5]:
scens1 = ['_TimersRandom_noWPcontrol', '_Timers9pm_noWPcontrol', '_Timers12am_noWPcontrol', '_TimersNone_noWPcontrol', '_TimersNone_WPcontrol_minpeak', '_TimersNone_WPcontrol_avgem']
scens2 = ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']

tables_dfs_timeofpeak = {penlevel: pd.DataFrame(np.zeros((7, 5)), 
                                                index=['_TimersRandom_noWPcontrol', '_Timers9pm_noWPcontrol', '_Timers12am_noWPcontrol', '_TimersNone_noWPcontrol', '_TimersNone_WPcontrol_minpeak', '_TimersNone_WPcontrol_avgem', '_TimersMixed_WPcontrol_minpeak'], 
                                                columns=['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork', 'BusinessAsUsual']) for penlevel in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}

for k, penlevel in enumerate([0.5, 1.0]):
#     if penlevel == 0.5:
    folder = 'Fuel1_Solar35_Wind3'
#     else:
#         folder = 'Fuel1_Solar35_Wind3_Curves'
    for i, scen1 in enumerate(scens1):
        for j, scen2 in enumerate(scens2):
            demand_df = pd.read_csv('Results/'+folder+'/fuel1_solar3.5_wind3_'+scen2+scen1+'_penlevel'+str(penlevel)+'_withstorage_dpdf_20220408.csv')
            inds = demand_df[pd.to_datetime(demand_df['datetime']).dt.weekday.isin([0,1,2,3,4])].index
            tables_dfs_timeofpeak[penlevel].loc[scen1, scen2] = np.where(demand_df.loc[inds, 'demand'].values.reshape(-1,24).mean(axis=0) == demand_df.loc[inds, 'demand'].values.reshape(-1,24).mean(axis=0).max())[0][0]
        
    scen2 = 'BusinessAsUsual'
    scen1 =  '_TimersMixed_WPcontrol_minpeak'
    i = i+1
    j = j+1
    demand_df = pd.read_csv('Results/'+folder+'/fuel1_solar3.5_wind3_'+scen2+scen1+'_penlevel'+str(penlevel)+'_withstorage_dpdf_20220408.csv')
    inds = demand_df[pd.to_datetime(demand_df['datetime']).dt.weekday.isin([0,1,2,3,4])].index
    tables_dfs_timeofpeak[penlevel].loc[scen1, scen2] = np.where(demand_df.loc[inds, 'demand'].values.reshape(-1,24).mean(axis=0) == demand_df.loc[inds, 'demand'].values.reshape(-1,24).mean(axis=0).max())[0][0]




In [17]:
print('Hour of peak in mean weekday *net* demand at 50%:')
tables_dfs_timeofpeak[0.5]

Hour of peak in mean weekday *net* demand at 50%:


,UniversalHome,HighHome,LowHome_HighWork,LowHome_LowWork,BusinessAsUsual
_TimersRandom_noWPcontrol,20.0,19.0,19.0,19.0,0.0
_Timers9pm_noWPcontrol,21.0,21.0,19.0,19.0,0.0
_Timers12am_noWPcontrol,19.0,19.0,19.0,19.0,0.0
_TimersNone_noWPcontrol,20.0,19.0,19.0,19.0,0.0
_TimersNone_WPcontrol_minpeak,20.0,19.0,19.0,19.0,0.0
_TimersNone_WPcontrol_avgem,20.0,19.0,19.0,19.0,0.0
_TimersMixed_WPcontrol_minpeak,0.0,0.0,0.0,0.0,20.0


In [18]:
print('Hour of peak in mean weekday *net* demand at 100%:')
tables_dfs_timeofpeak[1.0]

Hour of peak in mean weekday *net* demand at 100%:


,UniversalHome,HighHome,LowHome_HighWork,LowHome_LowWork,BusinessAsUsual
_TimersRandom_noWPcontrol,21.0,20.0,19.0,19.0,0.0
_Timers9pm_noWPcontrol,21.0,21.0,21.0,21.0,0.0
_Timers12am_noWPcontrol,0.0,0.0,19.0,19.0,0.0
_TimersNone_noWPcontrol,20.0,20.0,19.0,19.0,0.0
_TimersNone_WPcontrol_minpeak,20.0,20.0,18.0,19.0,0.0
_TimersNone_WPcontrol_avgem,20.0,20.0,19.0,19.0,0.0
_TimersMixed_WPcontrol_minpeak,0.0,0.0,0.0,0.0,21.0


In [6]:
scens1 = ['_TimersRandom_noWPcontrol', '_Timers9pm_noWPcontrol', '_Timers12am_noWPcontrol', '_TimersNone_noWPcontrol', '_TimersNone_WPcontrol_minpeak', '_TimersNone_WPcontrol_avgem']
scens2 = ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']

vals1 = np.zeros((10, 7, 5)) # peak mean weekday total demand
vals2 = np.zeros((10, 7, 5)) # time of day
tables_dfs1 = {penlevel: pd.DataFrame(np.zeros((7, 5)), 
                                     index=['_TimersRandom_noWPcontrol', '_Timers9pm_noWPcontrol', '_Timers12am_noWPcontrol', '_TimersNone_noWPcontrol', '_TimersNone_WPcontrol_minpeak', '_TimersNone_WPcontrol_avgem', '_TimersMixed_WPcontrol_minpeak'], 
                                     columns=['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork', 'BusinessAsUsual']) for penlevel in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
tables_dfs2 = {penlevel: pd.DataFrame(np.zeros((7, 5)), 
                                     index=['_TimersRandom_noWPcontrol', '_Timers9pm_noWPcontrol', '_Timers12am_noWPcontrol', '_TimersNone_noWPcontrol', '_TimersNone_WPcontrol_minpeak', '_TimersNone_WPcontrol_avgem', '_TimersMixed_WPcontrol_minpeak'], 
                                     columns=['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork', 'BusinessAsUsual']) for penlevel in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
for k, penlevel in enumerate([0.5, 1.0]):#0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]):
#     if penlevel == 0.5:
    folder = 'Fuel1_Solar35_Wind3'
#     else:
#         folder = 'Fuel1_Solar35_Wind3_Curves'
    for i, scen1 in enumerate(scens1):
        for j, scen2 in enumerate(scens2):
            demand_df = pd.read_csv('Results/'+folder+'/fuel1_solar3.5_wind3_'+scen2+scen1+'_penlevel'+str(penlevel)+'_withstorage_dpdf_20220408.csv')
            inds = demand_df[pd.to_datetime(demand_df['datetime']).dt.weekday.isin([0,1,2,3,4])].index
            vals1[k, i, j] = demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()
            vals2[k, i, j] = demand_df.loc[inds, 'demand'].values.reshape(-1,24).mean(axis=0).max()
            tables_dfs1[penlevel].loc[scen1, scen2] = demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()
            tables_dfs2[penlevel].loc[scen1, scen2] = demand_df.loc[inds, 'demand'].values.reshape(-1,24).mean(axis=0).max()
        
    scen2 = 'BusinessAsUsual'
    scen1 =  '_TimersMixed_WPcontrol_minpeak'
    i = i+1
    j = j+1
    demand_df = pd.read_csv('Results/'+folder+'/fuel1_solar3.5_wind3_'+scen2+scen1+'_penlevel'+str(penlevel)+'_withstorage_dpdf_20220408.csv')
    inds = demand_df[pd.to_datetime(demand_df['datetime']).dt.weekday.isin([0,1,2,3,4])].index
    vals1[k, i, j] = demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()
    vals2[k, i, j] = demand_df.loc[inds, 'demand'].values.reshape(-1,24).mean(axis=0).max()
    tables_dfs1[penlevel].loc[scen1, scen2] = demand_df.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()
    tables_dfs2[penlevel].loc[scen1, scen2] = demand_df.loc[inds, 'demand'].values.reshape(-1,24).mean(axis=0).max()


In [7]:
refval1 = noev_scenario.loc[inds, 'total_incl_noncombustion'].values.reshape(-1,24).mean(axis=0).max()
refval2 = noev_scenario.loc[inds, 'demand'].values.reshape(-1,24).mean(axis=0).max()

In [23]:
(100*((tables_dfs2[1.0] - refval2)/refval2))

,UniversalHome,HighHome,LowHome_HighWork,LowHome_LowWork,BusinessAsUsual
_TimersRandom_noWPcontrol,32.424116,21.775052,13.115553,14.124028,-100.00000
_Timers9pm_noWPcontrol,49.993686,39.872716,13.318819,15.177804,-100.00000
_Timers12am_noWPcontrol,35.788711,21.673002,12.887177,13.907306,-100.00000
_TimersNone_noWPcontrol,36.366527,26.381608,14.828624,15.885407,-100.00000
_TimersNone_WPcontrol_minpeak,36.586622,26.710114,17.952481,17.596285,-100.00000
_TimersNone_WPcontrol_avgem,36.367571,26.306359,14.284113,15.554230,-100.00000
_TimersMixed_WPcontrol_minpeak,-100.000000,-100.000000,-100.000000,-100.000000,27.11315


In [24]:
(100*((tables_dfs2[0.5] - refval2)/refval2))

,UniversalHome,HighHome,LowHome_HighWork,LowHome_LowWork,BusinessAsUsual
_TimersRandom_noWPcontrol,14.553096,9.694186,6.611485,7.120727,-100.000000
_Timers9pm_noWPcontrol,24.890641,16.561794,6.521179,6.996634,-100.000000
_Timers12am_noWPcontrol,12.630856,9.368636,6.496398,7.011081,-100.000000
_TimersNone_noWPcontrol,17.448005,12.143379,7.477203,8.014644,-100.000000
_TimersNone_WPcontrol_minpeak,17.577502,12.766932,8.931029,8.887124,-100.000000
_TimersNone_WPcontrol_avgem,17.448619,12.042273,7.201719,7.845757,-100.000000
_TimersMixed_WPcontrol_minpeak,-100.000000,-100.000000,-100.000000,-100.000000,11.710346


In [8]:
print('Max at 50%:', (100*((tables_dfs2[0.5] - refval2)/refval2)).max())

Max at 50%: UniversalHome       24.890641
HighHome            16.561794
LowHome_HighWork     8.931029
LowHome_LowWork      8.887124
BusinessAsUsual     11.710346
dtype: float64


In [9]:
print('Max at 100%:', (100*((tables_dfs2[1.0] - refval2)/refval2)).max())

Max at 100%: UniversalHome       49.993686
HighHome            39.872716
LowHome_HighWork    17.952481
LowHome_LowWork     17.596285
BusinessAsUsual     27.113150
dtype: float64


In [27]:
print("The Business As Usual scenario increases typical peak net demand by X times more than the Low Home High Work scenario:")
val1 = (tables_dfs2[0.5].loc['_TimersMixed_WPcontrol_minpeak', 'BusinessAsUsual'] - refval2)/refval2
val2 = (tables_dfs2[0.5].loc['_TimersNone_noWPcontrol', 'LowHome_HighWork'] - refval2)/refval2
print(val1)
print(val2)
print(np.round(val1/val2, 2))

print('---'*5)
print('With 100% EVs')

val1 = (tables_dfs2[1.0].loc['_TimersMixed_WPcontrol_minpeak', 'BusinessAsUsual'] - refval2)/refval2
val2 = (tables_dfs2[1.0].loc['_TimersNone_noWPcontrol', 'LowHome_HighWork'] - refval2)/refval2
print(val1)
print(val2)
print(np.round(val1/val2, 2))


The Business As Usual scenario increases typical peak net demand by X times more than the Low Home High Work scenario:
0.11710345905606802
0.07477202563092196
1.57
---------------
With 100% EVs
0.2711314963851631
0.1482862398208202
1.83


In [28]:
print("The Universal Home scenario with 9pm timers increases typical peak net demand by X times more than the Low Home High Work scenario:")
val1 = (tables_dfs2[0.5].loc['_Timers9pm_noWPcontrol', 'UniversalHome'] - refval2)/refval2
val2 = (tables_dfs2[0.5].loc['_TimersNone_noWPcontrol', 'LowHome_HighWork'] - refval2)/refval2
print(val1)
print(val2)
print(np.round(val1/val2, 2))

print('---'*5)
print('With 100% EVs')

val1 = (tables_dfs2[1.0].loc['_Timers9pm_noWPcontrol', 'UniversalHome'] - refval2)/refval2
val2 = (tables_dfs2[1.0].loc['_TimersNone_noWPcontrol', 'LowHome_HighWork'] - refval2)/refval2
print(val1)
print(val2)
print(np.round(val1/val2, 2))


The Universal Home scenario with 9pm timers increases typical peak net demand by X times more than the Low Home High Work scenario:
0.24890641332322427
0.07477202563092196
3.33
---------------
With 100% EVs
0.4999368599562716
0.1482862398208202
3.37
